In [2]:
#Subir PostA_sin_body PostQ_sin_body_ni_title a HDFS
# importar hdfs
from hdfs import InsecureClient

#conectarme
client = InsecureClient('http://localhost:9870/', user='usuarioFAI')

#Subir un archivo
subirPostsA = client.upload('/so/','/home/usuarioFAI/PostA_sin_body.csv')
subirPostsQ = client.upload('/so/','/home/usuarioFAI/PostQ_sin_body_ni_title.csv')

# Listar todos los archivos de una carpeta.
listado = client.list('/so/')
print(listado)

['PostA.csv', 'PostA_SinDuplicar.csv', 'PostA_sin_body.csv', 'PostO.csv', 'PostQ.csv', 'PostQ_SinDuplicar.csv', 'PostQ_sin_body_ni_title.csv', 'Posts.xml', 'Tags.csv', 'Tags.xml', 'Votes.csv', 'Votes.xml', 'Votes_SinDuplicar.csv', 'input', 'username.csv']


In [ ]:
#Limpieza con PYSPARK

In [3]:
#importar la libreria
from pyspark.sql import SparkSession

In [4]:
#abrir una sesion con un nombre
spark=SparkSession.builder.appName('pysarkpruebas').getOrCreate()

In [5]:
#leer archivo de PostQ (Questions) de HDFS
#header true es para que la primera fila sea considerada los atributos y no una tupla mas
#inferschema es para que tome los tipos de datos como son, no todo string..prueben poner false y luego imprimir el esquema
dfq = spark.read.csv('hdfs://127.0.0.1:9000/so/PostQ_sin_body_ni_title.csv', header=True,  inferSchema=False)
dfq.printSchema()
dfq.show(5)

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- PostTypeId: string (nullable = true)
 |-- AcceptedAnswerId: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- AnswerCount: string (nullable = true)

+---+---+----------+----------------+--------------------+--------------------+-----------+
|_c0| Id|PostTypeId|AcceptedAnswerId|        CreationDate|                Tags|AnswerCount|
+---+---+----------+----------------+--------------------+--------------------+-----------+
|  0|  1|         1|               2|2015-10-29T15:56:...|<.net><asp.net-we...|          1|
|  1|  3|         1|               9|2015-10-29T23:54:...|            <delphi>|          3|
|  2|  5|         1|             208|2015-10-30T01:15:...|       <c++><string>|          7|
|  3|  6|         1|             654|2015-10-30T01:36:...|      <c++><casting>|          2|
|  4|  8|         1|             442|2015-10-30T10:26:...|<google-ma

In [6]:
#mostrar la cantidad de Questions y las filas duplicadas
cant_preguntas = dfq.count()
print("La cantidad de preguntas son: ",cant_preguntas)

#eliminar preguntas duplicadas (solo si los resultados anteriores son diferentes)
dfq = dfq.dropDuplicates()
#dfq = dfq.distinct()

cant_preg_sin_duplicar=dfq.count()
print("Cantidad de filas diferentes luego del borrado: ",cant_preg_sin_duplicar)

La cantidad de preguntas son:  145623
Cantidad de filas diferentes luego del borrado:  145623


In [7]:
dfq.show(5)

+---+----+----------+----------------+--------------------+--------------------+-----------+
|_c0|  Id|PostTypeId|AcceptedAnswerId|        CreationDate|                Tags|AnswerCount|
+---+----+----------+----------------+--------------------+--------------------+-----------+
|434|1912|         1|            1943|2015-12-29T18:46:...|     <java><android>|          2|
|487|2076|         1|            2078|2016-01-05T07:36:...|   <java><thymeleaf>|          3|
|501|2123|         1|            2125|2016-01-06T00:20:...|<css3><sass><gulp...|          1|
|585|2441|         1|            2444|2016-01-12T19:35:...|<javascript><angu...|          2|
|783|3125|         1|            3151|2016-01-25T22:27:...|     <java><android>|          2|
+---+----+----------+----------------+--------------------+--------------------+-----------+
only showing top 5 rows



In [8]:
#print("Cantidad de filas diferentes: ",dfq.distinct().count())
cant_preg_filtradas = cant_preguntas - cant_preg_sin_duplicar
print ('La cantidad de preguntas filtradas es: ',cant_preg_filtradas)

La cantidad de preguntas filtradas es:  0


In [9]:
#leer archivo de PostA (Answers) de HDFS
dfa = spark.read.csv('hdfs://127.0.0.1:9000/so/PostA_sin_body.csv', header=True, inferSchema=False)
#mostrar el esquema
dfa.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- PostTypeId: string (nullable = true)
 |-- ParentId: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- Score: string (nullable = true)



In [10]:
#mostrar la cantidad de Answers y las filas duplicadas
cant_respuestas = dfa.count()
print("La cantidad de Respuestas son: ",cant_respuestas)

#eliminar respuestas duplicadas (solo si los resultados anteriores son diferentes)
#dfa = dfa.dropDuplicates()
dfa = dfa.distinct()
cant_resp_no_duplicadas = dfa.count()
print('Cantidad de filas diferentes luego del borrado: ', cant_resp_no_duplicadas)

La cantidad de Respuestas son:  159308
Cantidad de filas diferentes luego del borrado:  159308


In [11]:
dfa.show(5)
#print('Cantidad de filas diferentes: ', dfa.distinct().count())
cant_resp_filtradas = cant_respuestas - cant_resp_no_duplicadas
print ('La cantidad de respuestas filtradas es: ',cant_resp_filtradas)

+---+----+----------+--------+--------------------+-----+
|_c0|  Id|PostTypeId|ParentId|        CreationDate|Score|
+---+----+----------+--------+--------------------+-----+
|239| 478|         2|     468|2015-12-05T12:17:...|    8|
|308| 738|         2|     368|2015-12-08T15:33:...|    5|
|398| 950|         2|     161|2015-12-12T14:16:...|    4|
|565|1283|         2|    1270|2015-12-17T19:36:...|    2|
|628|1439|         2|    1438|2015-12-21T03:17:...|    1|
+---+----+----------+--------+--------------------+-----+
only showing top 5 rows

La cantidad de respuestas filtradas es:  0


In [12]:
#leer archivo de Votes de HDFS
dfv = spark.read.csv('hdfs://127.0.0.1:9000/so/Votes.csv', header=True, inferSchema=False)
#mostrar el esquema
dfv.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Id: string (nullable = true)
 |-- PostId: string (nullable = true)
 |-- VoteTypeId: string (nullable = true)
 |-- CreationDate: string (nullable = true)



In [13]:
dfv.show(5)
#mostrar la cantidad de Answers y las filas duplicadas
cant_votes = dfv.count()
print("La cantidad de Votos son: ",cant_votes)

#eliminar respuestas duplicadas (solo si los resultados anteriores son diferentes)
#dfv = dfv.dropDuplicates()
dfv = dfv.distinct()

cant_votes_no_duplicadas = dfv.count()
print('Cantidad de filas diferentes luego del borrado: ', cant_votes_no_duplicadas)

+---+---+------+----------+--------------------+
|_c0| Id|PostId|VoteTypeId|        CreationDate|
+---+---+------+----------+--------------------+
|  0|  1|     1|         2|2015-10-29T00:00:...|
|  1|  2|     2|         2|2015-10-29T00:00:...|
|  2|  3|     3|         2|2015-10-30T00:00:...|
|  3|  4|     5|         2|2015-10-30T00:00:...|
|  4|  5|     7|         2|2015-10-30T00:00:...|
+---+---+------+----------+--------------------+
only showing top 5 rows

La cantidad de Votos son:  628897
Cantidad de filas diferentes luego del borrado:  628897


In [14]:
#print('Cantidad de filas diferentes: ', dfa.distinct().count())
cant_votes_filtradas = cant_votes - cant_votes_no_duplicadas
print ('La cantidad de respuestas filtradas es: ',cant_votes_filtradas)

La cantidad de respuestas filtradas es:  0


In [15]:
import pandas as pd

Titulos = ['cantidad preguntas', 'cantidad preguntas sin duplicar', 'cantidad preguntas filtradas', 
        'cantidad respuestas', 'cantidad respuestas sin duplicar', 'cantidad respuestas filtradas',
          'cantidad votos','cantidad votos sin duplicar','cantidad votos filtrados']
Cantidades = []

Cantidades.append(cant_preguntas) #asi con cada cant
Cantidades.append(cant_preg_sin_duplicar) #asi con cada cant
Cantidades.append(cant_preg_filtradas)
Cantidades.append(cant_respuestas) #asi con cada cant
Cantidades.append(cant_resp_no_duplicadas) #asi con cada cant
Cantidades.append(cant_resp_filtradas)
Cantidades.append(cant_votes)
Cantidades.append(cant_votes_no_duplicadas) #asi con cada cant
Cantidades.append(cant_votes_filtradas)

df = pd.DataFrame({'Descripción':Titulos, 'Cantidades':Cantidades})

#df.show() #verificar si funciona

print(df)
#lo guardamos 
df.to_csv('resumen_cantidades.csv') 

                        Descripción  Cantidades
0                cantidad preguntas      145623
1   cantidad preguntas sin duplicar      145623
2      cantidad preguntas filtradas           0
3               cantidad respuestas      159308
4  cantidad respuestas sin duplicar      159308
5     cantidad respuestas filtradas           0
6                    cantidad votos      628897
7       cantidad votos sin duplicar      628897
8          cantidad votos filtrados           0


In [16]:
#Guardamos PostA_SinDuplicar.csv
dfa.toPandas().to_csv('PostA_SinDuplicar.csv')

In [17]:
#Guardamos PostA_SinDuplicar.csv
dfq.toPandas().to_csv('PostQ_SinDuplicar.csv')

In [18]:
#Guardamos Votes_SinDuplicar.csv
dfv.toPandas().to_csv('Votes_SinDuplicar.csv')